In [2]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import pandas as pd

In [3]:

model = BertForSequenceClassification.from_pretrained("fine_tuned_model", num_labels=3)
tokenizer = BertTokenizer.from_pretrained("fine_tuned_model")

device = torch.device("cpu")
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [4]:
data = pd.read_csv('all_data.csv')
data

,comment,label
0,좌배 까는건 ㅇㅂ,0
1,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,0
2,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,0
3,731부대의 후예라 그런지 가학적인 아이디어는 세계최고임 이래서 애교만 떨어도 돈 ...,0
4,재앙이한건햇노,2
...,...,...
16431,성소수자 이빈 이띤 좆같고 뱡신 같은 말은 누가 만드는거냐?,1
16432,갱상도가아니라 홍어지 개좃같은 홍어년들,2
16433,말레이시아랑 인도네시아 여자 존나 못생겼던데,1
16434,링크도 안박고 가서 글 쓰자고 선동하네,0


In [5]:
data['pred_0'] = None
data['pred_1'] = None
data['pred_2'] = None

data

,comment,label,pred_0,pred_1,pred_2
0,좌배 까는건 ㅇㅂ,0,None,None,None
1,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,0,None,None,None
2,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,0,None,None,None
3,731부대의 후예라 그런지 가학적인 아이디어는 세계최고임 이래서 애교만 떨어도 돈 ...,0,None,None,None
4,재앙이한건햇노,2,None,None,None
...,...,...,...,...,...
16431,성소수자 이빈 이띤 좆같고 뱡신 같은 말은 누가 만드는거냐?,1,None,None,None
16432,갱상도가아니라 홍어지 개좃같은 홍어년들,2,None,None,None
16433,말레이시아랑 인도네시아 여자 존나 못생겼던데,1,None,None,None
16434,링크도 안박고 가서 글 쓰자고 선동하네,0,None,None,None


In [6]:
for idx, row in data.iterrows():
    sentence = row['comment']
    encoded_input = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        padding='max_length',
        max_length=128,
        truncation=True,
        return_tensors='pt'
    )
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        #predicted_labels = torch.argmax(logits, dim=1)
    data.iat[idx,2] = logits[0][0].item()
    data.iat[idx,3] = logits[0][1].item()
    data.iat[idx,4] = logits[0][2].item()
    

In [7]:
data

,comment,label,pred_0,pred_1,pred_2
0,좌배 까는건 ㅇㅂ,0,2.127698,-1.259958,-1.049847
1,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,0,2.056225,-0.722713,-1.635577
2,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,0,1.605559,-0.812442,-0.89549
3,731부대의 후예라 그런지 가학적인 아이디어는 세계최고임 이래서 애교만 떨어도 돈 ...,0,-1.197054,1.218151,0.053495
4,재앙이한건햇노,2,1.217642,-0.54542,-0.628623
...,...,...,...,...,...
16431,성소수자 이빈 이띤 좆같고 뱡신 같은 말은 누가 만드는거냐?,1,0.509378,2.662991,-3.048843
16432,갱상도가아니라 홍어지 개좃같은 홍어년들,2,-1.944761,-2.865839,4.90368
16433,말레이시아랑 인도네시아 여자 존나 못생겼던데,1,0.686432,1.920945,-2.620282
16434,링크도 안박고 가서 글 쓰자고 선동하네,0,2.966615,-0.987901,-2.115355


In [8]:
data.to_csv("all_data_pred.csv",encoding = 'cp949', index = False)

In [10]:
import OpenMax as om

In [11]:
ori_label = data['label']
pred_label = list(zip(data['pred_0'],data['pred_1'],data['pred_2']))
pred_label = [[a,b,c] for a,b,c in pred_label]
print(ori_label[:10])
print(pred_label[:10])

0    0
1    0
2    0
3    0
4    2
5    0
6    0
7    0
8    0
9    0
Name: label, dtype: int64
[[2.1276981830596924, -1.259958267211914, -1.0498474836349487], [2.056225061416626, -0.7227131724357605, -1.6355770826339722], [1.6055593490600586, -0.8124424815177917, -0.895490288734436], [-1.197054386138916, 1.2181510925292969, 0.05349519103765488], [1.2176417112350464, -0.5454196333885193, -0.6286226511001587], [3.075437307357788, 0.42066535353660583, -3.6449761390686035], [2.464428663253784, -0.8731300234794617, -1.83771550655365], [1.2422901391983032, 1.7102365493774414, -3.067295551300049], [2.089521884918213, 0.04876619949936867, -1.9609898328781128], [2.945996046066284, -0.14480488002300262, -2.8641138076782227]]


In [12]:
openmax = om.OpenMax()
openmax.fit(pred_label,ori_label)

In [13]:
def predict_sentence(sentence):
    encoded_input = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        padding='max_length',
        max_length=128,
        truncation=True,
        return_tensors='pt'
    )
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = [0,0,0]
    logits[0] = outputs.logits[0][0].item()
    logits[1] = outputs.logits[0][1].item()
    logits[2] = outputs.logits[0][2].item()
    
    om_result = openmax.calculate(logits)
    _class = ['혐오','성차별','일베']
    print("입력문장 : ",sentence)
    print()
    print("모델 예측 결과 - OpenMax 미적용")
    print("혐오 : ",logits[0])
    print("성차별 : ",logits[1])
    print("일베 : ",logits[2])
    print()
    print("모델 예측 결과 - OpenMax 적용")
    print("Unkown : ",om_result[0])
    print("혐오 : ",om_result[1])
    print("성차별 : ",om_result[2])
    print("일베 : ",om_result[3])
    
   

In [30]:
predict_sentence("남자가 쪼잔하게")

입력문장 :  남자가 쪼잔하게

모델 예측 결과 - OpenMax 미적용
혐오 :  0.22245274484157562
성차별 :  1.513607382774353
일베 :  -1.7779442071914673

모델 예측 결과 - OpenMax 적용
Unkown :  0.1437159900117854
혐오 :  0.1793704001525054
성차별 :  0.6526379560422231
일베 :  0.02427565379348602


In [24]:
openmax.calculate([1,1,1])

[0.11069866539568819,
 0.29422751187099094,
 0.2975303280194986,
 0.2975434947138223]